In [8]:
'''Parses the proteins into epitopes by cutting them up over sliding windows'''

'Parses the proteins into epitopes by cutting them up over sliding windows'

In [9]:
import gzip
from Bio import SeqIO
from Bio.Seq import Seq
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [11]:

# with open('../data/raw/SARS_proteins.fasta', "rt") as handle:
with open('data/SARS_proteins.fasta', "rt") as handle:
    records = list(SeqIO.parse(handle, "fasta"))

In [12]:
# type(records) # list
# len(records) # 15
# type(records[0]) # Bio.SeqRecord.SeqRecord      class instance
# records[0].description
records[0]
# records[0].description.split('GN=')[-1].split(' ')[0] # 1a

SeqRecord(seq=Seq('MESLVLGVNEKTHVQLSLPVLQVRDVLVRGFGDSVEEALSEAREHLKNGTCGLV...FAV', SingleLetterAlphabet()), id='sp|P0C6U8|R1A_CVHSA', name='sp|P0C6U8|R1A_CVHSA', description='sp|P0C6U8|R1A_CVHSA Replicase polyprotein 1a OS=Human SARS coronavirus (SARS-CoV) (Severe acute respiratory syndrome coronavirus) OX=694009 GN=1a PE=1 SV=1', dbxrefs=[])

In [13]:
records[3].description.split('GN=')[-1].split(' ')[0] # = 'N', it is the GN = ... in the description of the Bio.SeqRecord.SeqRecord class


'N'

In [14]:
for i in range(len(records)):
    print(len(records[i].seq), records[i].description.split('GN=')[-1].split(' ')[0])

4382 1a
7073 rep
1255 S
422 N
221 M
274 3a
154 3b
63 6
122 7a
98 9b
76 E
39 8a
44 7b
70 ORF14
84 8b


In [15]:
records[i] # same as records[14], since records has length 15


SeqRecord(seq=Seq('MCLKILVRYNTRGNTYSTAWLCALGKVLPFHRWHTMVQTCTPNVTINCQDPAGG...RTN', SingleLetterAlphabet()), id='sp|Q80H93|NS8B_CVHSA', name='sp|Q80H93|NS8B_CVHSA', description='sp|Q80H93|NS8B_CVHSA Non-structural protein 8b OS=Human SARS coronavirus (SARS-CoV) (Severe acute respiratory syndrome coronavirus) OX=694009 GN=8b PE=4 SV=1', dbxrefs=[])

In [16]:
### Breaking down the larger '1a' and 'rep' proteins

import copy 
for_glyc_pred = []
for i in range(len(records)):
    protein= records[i].description.split('GN=')[-1].split(' ')[0]
    if protein == '1a' or protein == 'rep':
        sars1 = copy.deepcopy(records[i])
        sars1.seq = sars1.seq[:3500]
        sars1.description = 'FIRST_3500_'+sars1.description
        sars1.name = 'FIRST_3500_'+sars1.name
        sars1.id = 'FIRST_3500_'+sars1.id
        
        ### It's not exactly the last 3470, but rather the 3470th peptide until the end
        ### and has length 912 for '1a' and 3603 for 'rep'
        sars2 = copy.deepcopy(records[i])
        sars2.seq = sars2.seq[3470:]
        sars2.description = 'LAST_3470_ONWARDS_'+sars2.description
        sars2.name = 'LAST_3470_ONWARDS_'+sars2.name
        sars2.id = 'LAST_3470_ONWARDS_'+sars2.id
        
        for_glyc_pred.append(sars1)
        for_glyc_pred.append(sars2)
    else: 
        for_glyc_pred.append(records[i])
        

In [17]:

for_glyc_pred[0]

SeqRecord(seq=Seq('MESLVLGVNEKTHVQLSLPVLQVRDVLVRGFGDSVEEALSEAREHLKNGTCGLV...GIA', SingleLetterAlphabet()), id='FIRST_3500_sp|P0C6U8|R1A_CVHSA', name='FIRST_3500_sp|P0C6U8|R1A_CVHSA', description='FIRST_3500_sp|P0C6U8|R1A_CVHSA Replicase polyprotein 1a OS=Human SARS coronavirus (SARS-CoV) (Severe acute respiratory syndrome coronavirus) OX=694009 GN=1a PE=1 SV=1', dbxrefs=[])

In [18]:
from Bio import SeqIO

### the file SARS_for_glyc.fasta does not exist
with open("../data/processed/SARS_for_glyc.fasta", "w") as output_handle:
    SeqIO.write(for_glyc_pred, output_handle, "fasta")

In [21]:
# len(all_windows_startpos)

In [22]:
# all_windows_startpos[20]

In [20]:
# len(all_windows)

In [23]:
window_sizes = list(range(8,26))
for i_rec in range(len(records)):
    seq = str(records[i_rec].seq)
    idd = records[i_rec].description.split('GN=')[-1].split(' ')[0]
    all_windows = []
    all_windows_startpos = []
    for window in window_sizes:
        epitopes = []
        start_pos = []
        for i in range(len(seq)-window+1):
            epitopes.append(seq[i:i+window] )
            start_pos.append(i)
            
        # protein, epitopes, length
        all_windows += epitopes
        all_windows_startpos+=start_pos
        
        
    df_temp = pd.DataFrame( all_windows )
    df_temp.columns = ['Epitope']
    df_temp['Protein'] = idd
    df_temp['start_pos'] = np.asarray(all_windows_startpos)
    #display(df_temp)
    #print(i)
    if i_rec==0:
        print('hey')
        df_epi = df_temp
    else: 
        df_epi = df_epi.append(df_temp) #pd.concat([df_epi, df_temp], axis=0, ignore_index=True)
        # df_epi = df_epi.append(df_temp, ignore_index = True), would clear up the indexing of the dataframe
        
df_epi.head()

hey


,Epitope,Protein,start_pos
0,MESLVLGV,1a,0
1,ESLVLGVN,1a,1
2,SLVLGVNE,1a,2
3,LVLGVNEK,1a,3
4,VLGVNEKT,1a,4


In [24]:
df_epi

,Epitope,Protein,start_pos
0,MESLVLGV,1a,0
1,ESLVLGVN,1a,1
2,SLVLGVNE,1a,2
3,LVLGVNEK,1a,3
4,VLGVNEKT,1a,4
...,...,...,...
1228,LIARCWYLHEGHQTAAFRDVLVVLN,8b,55
1229,IARCWYLHEGHQTAAFRDVLVVLNK,8b,56
1230,ARCWYLHEGHQTAAFRDVLVVLNKR,8b,57
1231,RCWYLHEGHQTAAFRDVLVVLNKRT,8b,58


In [25]:
df_epi.shape

(254601, 3)

In [26]:
# df_temp.shape
df_temp.head()

,Epitope,Protein,start_pos
0,MCLKILVR,8b,0
1,CLKILVRY,8b,1
2,LKILVRYN,8b,2
3,KILVRYNT,8b,3
4,ILVRYNTR,8b,4


In [27]:
df_epi.Protein.unique()

array(['1a', 'rep', 'S', 'N', 'M', '3a', '3b', '6', '7a', '9b', 'E', '8a',
       '7b', 'ORF14', '8b'], dtype=object)

In [21]:
df_epi.to_csv('../data/processed/SARS_all_protein_epitopes.csv', index=False)